# Point Cloud Registration

## Problem Definition

### Basic Notation

We know the locations of the phantom grid intersections.  We denote this set of points as $A$ where $\forall \mathbf{a} \in A, \mathbf{a} \in \mathbb{R}^3$.  The position of the origin may be arbitrarily defined.  We assume that the points in $A$ form a Cartesian grid with even spacing along each dimension, $\Delta$, and that the axes of the coordinate system line up with the grid.  Thus we have

$$
\forall \mathbf{a} \in A \;\exists\; i, j, k \in \mathbb{Z} \; \textrm{such that} \; \mathbf{a} = \begin{bmatrix} i \Delta \\ j \Delta \\ k \Delta \end{bmatrix}.
$$

The feature-matching stage of the geometric distortion algorithm takes a 3D MRI data set and returns a set of points, which we denote as $B$, where $\forall \mathbf{b} \in B, \mathbf{b} \in \mathbb{R}^3$.  The isocenter of the scanner defines the origin of $B$'s coordinate system. 

The frame of reference of $A$ and $B$ are likely different, thus we must register the two frames of reference.  In particular, we are looking to determine the six free variables, $x$, $y$, $z$, $\theta$, $\phi$, and $\xi$ which define the affine transformation matrix that can relate the two frames of reference.  We define this matrix as

$$
\mathrm{S} = \mathrm{R}_x(\theta) \cdot \mathrm{R}_y(\phi) \cdot \mathrm{R}_z(\xi) \cdot \mathrm{T}(x, y, z)
$$

where

$$
\begin{align*}
\mathrm{R}_x(\theta) &=
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & \cos(\theta) & -\sin(\theta) & 0\\
0 & \sin(\theta) & \cos(\theta) & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix},\\
\mathrm{R}_y(\phi) &=
\begin{bmatrix}
\cos(\phi) & 0 & \sin(\phi) & 0\\
0 & 1 & 0 & 0\\
-\sin(\phi) & 0 & \cos(\phi) & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix},\\
\mathrm{R}_z(\xi) &=
\begin{bmatrix}
\cos(\phi) & \sin(\phi) & 0 & 0\\
-\sin(\phi) & \cos(\phi) & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix}, \textrm{and}\\
\mathrm{T}(x, y, z) &=
\begin{bmatrix}
1 & 0 & 0 & x\\
0 & 1 & 0 & y\\
0 & 0 & 1 & z\\
0 & 0 & 0 & 1\\
\end{bmatrix}.\\
\end{align*}
$$

We define the set of grid points $A$ that have been transformed into $B$'s frame of reference to be

$$
\mathrm{A}_\textrm{S} = \left\{ \mathbf{a}_\textrm{S} \;\; \Big| \;\; \begin{bmatrix} \mathbf{a}_\textrm{S} \\ 1 \end{bmatrix} = \mathrm{S} \cdot \begin{bmatrix} \mathbf{a} \\ 1 \end{bmatrix} , \;\; \mathbf{a} \in A \right\}
$$

It is useful to categorize the points in $A$ and $B$.

- We call the points in $B$ that correspond to points in $A$ matching points.
- We call the points in $B$ that do not correspond to points in $A$ false positives.
- We call the points in $A$ that do not have a corresponding point in $B$ false negatives.

### Simplifying Assumptions

We have some additional priori knowledge about the problem that makes it solvable.  We number these assumptions for easy reference:

**Assumption 1**: We can neglect any manufacturing error in the points of $A$.

**Assumption 2**: We can rely on the first stage of our algorithm being "reasonably good".  In other words, we can assume that some fraction, $\gamma$, of the points in $B$ are matched points. We assumed that $\gamma > 0.8$.

**Assumption 3**: The displacement between matched points, $|\mathbf{b} - \mathbf{a}_\textrm{S}|$ is bounded for a given distance from the isocenter.  In particular, the closer a pair of points are to the isocenter, the closer they must be in order to be considered a "matching pair".  The function that defines this cutoff we call $\rho(|\mathbf{b}|)$.  We suspect that even at the extremities of the phantom, the matching cutoff will probably be less than the grid spacing, i.e. $\rho(|\mathbf{b}_\textrm{max}|) \approx \Delta/2$, although we should do some more background research into this.

Note that the displacement between matching points has several components.  The geometric distortion introduced by the MRI is what we are attempting to measure, however there are other contributions to the displacement.  In particular, the first stage of our algorithm will likely introduce additional errors.

**Assumption 4**: There is no systematic translational geometric distortion in the points $B$.  For example, it is feasible that the MRI shifts all points over by 1mm.  Such an error would be indistinguishable from a registration error, and thus we will not be able to detect it.  It does not seem to have any clinical consequence.  Thus, the mean distortion across all the points will be zero.

**Assumption 5**: Only a small amount of rotation will be required to register $A$ and $B$.  E.g. $|\theta| < \theta_m^\circ \approx 5^\circ$, $|\phi| < \phi_m \approx 15^\circ$, and $|\xi| < \xi_m \approx 5^\circ$.

This assumption is important. $A$ will typically be roughly symmetric with respect to one or more axis.  If we did not constrain the magnitude of the rotations our problem would have several local minimums of nearly equal depth.  For example, imagine $A$ is a 16x16x16cube of grid points, with a couple points missing along one edge of the cube.  Further, assume that $B$ has no false positives, and that it has a full set of "matching points"--i.e. the first stage of our algorithm was really good and found all 16x16x16 - 2 points!  Rotating the set of points along any axis of the grid will likely result in a close match (only the two points will not be overlapping).

In order to avoid solving such an ill-posed problem, we simply rely on the technicians doing a reasonably good job aligning the phantom properly.  Rotation along the $x$ or $z$ directions will be constrained by the MRI table, and thus will be quite small.  Rotation along the $y$ dimension will be more likely.

### Formulating the Optimization Problem

We want to find the "best" registration matrix S that registers the two sets of points. In order to formulate the problem so that we can provide a function to a mathematical optimizer, we need to mathematically define what it means to be the "best".

The "best" matrix $\mathrm{S}(x, y, z, \theta, \phi, \xi)$ will fulfill the following properties.

- It will minimize the distance between matching points in $A_\mathrm{S}$ and $B$.  Note we are minimizing the distance, instead of the distance-squared, because we know that there will be a certain amount of displacement between the matched points.  If we minimized the "distance-squared" between matching points, this would likely give too much importance to points with significant geometric distortion.
- It will maximize the number of matching points.  If we did not have this condition we could either not match any points or we could match a set of grid points that are shifted over by one or more $\Delta$s.
- It will weight the distance between matching points near the isocenter more than points further away from the isocenter--the relative weighting can be characterized by a monotonically decreasing function $g$.
- It will ignore any "false positives", where a false positive is defined as a point in $\mathbf{b} \in B$ such that is $|\mathbf{b} − \mathbf{a}_{\textrm{S},\textrm{min}}| < \rho(|\mathbf{b}|)$ where $\mathbf{a}_{\textrm{S},\textrm{min}}$ is the nearest unmatched point to $\mathbf{b}$.  Note that if $\rho < \Delta/2$, it would be impossible for a given point in $A$ to match to two different points in $B$.  However, if $\rho >= \Delta/2$, it is possible for a point in $A$ to be the closest point to two points in $B$.  In other words, it has the potential to be matched to two different points in $B$.  When this occurs, we match it to the closest point in $B$, and match the other point in $B$ to the next closest potentially matching point in $A$, if one exists, and leave it unmatched otherwise.

We can find $\mathrm{S}$ by minimizing the following function:

$$
f(x, y, z, \theta, \phi, \xi) =
\sum_A g(\left|\mathbf{b}\right|)
\;
(\left|\mathbf{b} - \mathbf{a}_{\textrm{S},\textrm{min}}\right| - \rho(|\mathbf{b}|))
\;
u(\rho(|\mathbf{b}|) - \left|\mathbf{b} - \mathbf{a}_{\textrm{S},\textrm{min}}\right|)
$$

where $u$ is the Heaviside function.  Note that we have not explicitly defined $g$ or $\rho$.  More research will need to be done to determine what the best form for both of these functions will be.  Ideally, their form will be rooted in our knowledge of MRI physics.